# Encontro 09: Simulação de Balanceamento

Importando a biblioteca:

In [ ]:
import sys
sys.path.append('..')

import socnet as sn

Configurando a biblioteca:

In [ ]:
sn.node_size = 3
sn.node_color = (0, 0, 0)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

Gerando um grafo completo:

In [ ]:
g = sn.generate_complete_graph(15)

sn.show_graph(g)

Atribuindo aleatoriamente tipos negativos e positivos às arestas:

In [ ]:
from random import random


def randomize_types(g):
    for n, m in g.edges():
        if random() < 0.5:
            g.edge[n][m]['type'] = -1
        else:
            g.edge[n][m]['type'] = 1


randomize_types(g)

Convertendo tipos em cores para visualização:

In [ ]:
def convert_types_to_colors(g):
    for n, m in g.edges():
        if g.edge[n][m]['type'] == -1:
            g.edge[n][m]['color'] = (255, 0, 0)
        else:
            g.edge[n][m]['color'] = (0, 0, 255)


convert_types_to_colors(g)

sn.show_graph(g)

Definindo uma atualização de tipo:

In [ ]:
# Esta função calcula a pressão que cada aresta sofre para mudar seu tipo,
# ou seja, a quantidade de triângulos instáveis aos quais ela pertence.
# Se nenhuma aresta está sofrendo pressão, a função devolve False. Senão,
# a aresta com maior pressão muda de tipo e a função devolve True.

def update_type(g):

    # Inicializa as pressões.

    for n, m in g.edges():
        g.edge[n][m]['pressure'] = 0

    # Para cada triângulo do grafo.

    for n in g.nodes():
        for m in g.nodes():
            if n != m:
                for l in g.nodes():
                    if n != l and m != l:

                        # Armazena em uma lista as três arestas do triângulo.

                        edges = [(n, m), (n, l), (m, l)]

                        # Conta quantas das arestas do triângulo são positivas.

                        positives = 0

                        for e in edges:
                            if g.edge[e[0]][e[1]]['type'] == 1:
                                positives += 1

                        # Se há zero ou duas positivas, incrementa as pressões.

                        if positives == 0 or positives == 2:
                            for e in edges:
                                g.edge[e[0]][e[1]]['pressure'] += 1

    # Obtém a aresta com maior pressão.

    n, m = max(g.edges(), key=lambda e: g.edge[e[0]][e[1]]['pressure'])

    # Se essa maior pressão não for zero, muda o tipo da aresta e devolve True.

    if g.edge[n][m]['pressure'] > 0:
        g.edge[n][m]['type'] *= -1

        return True

    # Se essa maior pressão for zero, devolve False.

    return False

Construindo uma animação que simula o que acontece quando tipos são mudados até as pressões acabarem.

In [ ]:
def snapshot(g, frames):
    convert_types_to_colors(g)

    frame = sn.generate_frame(g)

    frames.append(frame)


frames = []

# Atribui aleatoriamente tipos negativos e positivos às arestas.
randomize_types(g)

# Atribui aleatoriamente posições aos nós.
sn.randomize_positions(g)

snapshot(g, frames)

# Continua até que update_type devolva False, indicando que as pressões acabaram.
while update_type(g):

    # Move um pouco os vértices de posição, usando o atributo 'type' das
    # arestas como referência para saber o quanto dois vértices se atraem.
    sn.update_positions(g, 'type')

    snapshot(g, frames)

sn.show_animation(frames)

Balanceamento converge para polarização!